Iniciamos importando las librerías necesarios para nuestro código.

In [ ]:
import requests
import json
import time

Definimos algunas variables globales, en este caso `bearer_token` para acceder al servicio de Twitter y `search_url` como el enlace base para realizar las peticiones por medio del método `search`.



In [ ]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAANYRagEAAAAAZMp6MHxLFIKjb1%2B9nkrGURy1%2FOs%3D60MbAdCFxfsTE7nTl2b8PlSGYdZj0huPg7RbMG4z21l4e0jReV'
search_url = 'https://api.twitter.com/2/tweets/search/all'


Se define `query_params` que alberga el query que permite definir qué tipo de información se quiere extraer.


*   [Enlace a la documentación para construir queries](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query) 
*   [Lista de operadores](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#list)
*   [Operadores booleanos y agrupación](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#boolean)
*   [Algunos ejemplos...](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#examples)



Requests permite definir enlaces web de una manera sencilla. Solo se necesita crear un diccionario con los parámetros que se quieran pasar al enlace:

In [ ]:
query_params = {'query': '(ukraine OR #ukraine) lang:en -is:retweet',
'tweet.fields': 'author_id,created_at,public_metrics',
'start_time': '2022-02-22T06:00:00Z',
'end_time': '2022-02-23T06:00:00Z',
'max_results': 500}

Se definen funciones para manejar algunas actividades:


*   Crear el encabezado base
*   Crear la conexión con Twitter
*   Manejar la conexión y la descarga de datos



In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [ ]:
def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        print('--->>> There was an error! <<<---')
    return response.json()

In [ ]:
def main(file_name):
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(search_url, headers, query_params)
    # ----- escribe los datos en un archivo de texto -----
    # itera sobre la llave 'data' del diccionario devuelto, el cual es una lista
    for x in json_response['data']:
        print(x) # opcional si se quiere ver la info en pantalla
        with open(file_name, 'a') as f:
            f.write(json.dumps(x)+'\n')
    # revisa si existe 'next_token' en el objeto json devuelto para paginar
    try:
        next_token = json_response['meta']['next_token']
        query_params['next_token'] = next_token
        # recursion: a la siguiente pagina del request
        time.sleep(5)
        main(file_name)
    except Exception as e:
        print('No next token')
        pass

Finalmente, ejecutar el código especificando el nombre del archivo donde se desea guardar la información.

In [ ]:
file_name = '' #el nombre del archivo donde se guardan los tweets en formato JSON
main(file_name)